# Data wrangling and cleaning:

Two data sets will be created; one with candidate financial data from Propublica, the other with the representatives for each congressional district taken from Wikipedia. The Propublica data set will be updated to represent winners and losers as per the wikipedia data set. A few records from the Propublica data set will need to be updated by hand as there were several duplicate records and two that did not match up from the Wikipedia data set.

# 1. Wrangle Propublica API Data

Two functions were created that would be applied over a list of all 50 states to compile finacial data for all candidates in the 2020 congressional election. The function st() makes an initial request to Propublica to get a list of all the candidates who ran. The function then iterates over that list, making repeated requests to Propublica for each individual candidate's financial data. A pandas data frame is returned. An additional function, er(), is utiized by st() to catch any errors encountered. If er() encounters an error, the function calls itself again, recursively, or, returns the request if no error is encountered.  


In [9]:
import requests, json; import pandas as pd
from bs4 import BeautifulSoup

In [1]:
def er(url, head): # error catch, recursive
    global r; r = requests.get(url, headers = head)
    if r.status_code != 200: er(url, head) 
    if r.text[11:13] != 'OK': er(url, head)
    return r


def st(state): #get list of candidates for each state and then each candidates' info.
    df = pd.DataFrame(columns=['st', 'dist', 'name', 'id', 'party', 'total', 'pac', 'win'])
    u = 'https://api.propublica.org/campaign-finance/v1/2020'
    h = {'X-API-Key': '1-800-API-KEYS-R-US'} # dummy api key
    cans = er(u+'/races/%s/house.json' %state, h).json()['results'] # get list of candidates who ran in each state
    
    for i in range(len(cans)): # enter list into data frame
        df = df.append({'st': state,
                        'name': cans[i]['candidate']['name'].lower(), 
                        'id': cans[i]['candidate']['id'], 
                        'party':cans[i]['candidate']['party'],},ignore_index = True)
        
    for j in range(len(df)):
        can_data = er(u+'/candidates/%s.json' %(df.iloc[j,3]), h).json() # get individual candidate info
        district = can_data['results'][0]['district']
        df.iat[j,1] = 98 if district == None else 99 if not district[16:18].isnumeric() else int(district[16:18])
        df.iat[j,5] = int(can_data['results'][0]['total_contributions'])
        df.iat[j,6] = int(can_data['results'][0]['total_from_pacs'])
    
    return df


States that are 'at-large' have no districts and the request to Propublica returns 'None' for the district; these were marked as '98' in the data frame. Also, if there is an error in the candidate's district, '99' was entered into the data frame.  

In [2]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 
    'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 
    'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [3]:
#iterate over all 50 states, call st() for each state, concatenate to df1
df1 = st(states[0])
for i in range(1, len(states)): df1 = pd.concat([df1, st(states[i])])

In [4]:
df1.to_csv('propub_csv_final', index = None, header=True) #save CSV 

# 2. Wrangle Wikipedia Table

The function wiki() scrapes a Wikipedia page listing all 435 congressional districts and their representatives and returns a data frame. The function abbreviate() changes each state's full name to its abbreviation so as to match the Propublica data. Also, the function spanish() removes any accented characters so as to better match Propublica's data.

As some candidates have complicated names, such as George Joseph "Mike" Kelly Jr. from PA 16, a search was conducted on the FEC website for each candidate's FEC ID. This will aid in matching the Propublica candidates to the Wikipedia candidates as the names are not perfect matches. Three functions were utilized. url_nm() parses the candidate's name and returns an appropriate string for the FEC website's search feature. ids() scrapes the results of the page returned from the FEC name search and returns the ID. wiki_id() iterates over each name in the Wikipedia data frame.


In [5]:
def abbreviate(x): #converts state's full name to abbreviation
    abv = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 
     'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 
     'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 
     'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
     'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 
     'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
     'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 
     'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 
     'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}
    a = x.split(); b = len(a)
    return abv[a[0]] if b == 2 else abv[' '.join([a[0], a[1]])]

def spanish(x): #converts accented characters to non-accented
    sp = {'á': 'a', 'é': 'e', 'í': 'i', 'ú': 'u'}
    return ''.join(sp[i] if i in sp.keys() else i for i in list(x))
        

def wiki(): #scrape Wikipedia table of congressional districts' representatives
    df = pd.DataFrame(columns=['st', 'dist', 'st/dist','name','party', 'id'])
    url = 'https://en.wikipedia.org/wiki/List_of_current_members_of_the_United_States_House_of_Representatives'
    s = BeautifulSoup(requests.get(url).text, 'html.parser')
    rows = s.find('table',{'class': 'wikitable sortable', 'id':'votingmembers'}).tbody.find_all('tr')
    
    for i in range(1, len(rows)):
        tds = rows[i].find_all('td')
        if len(tds) == 9:
            values = [tds[0].text.replace('\n', ''), spanish(tds[1].text.replace('\n', '').lower()), 
                      tds[3].text.replace('\n', '')]
        else: values = [tds[0].text.replace('\n', ''), 'vacant', 'vacant']
        df = df.append(pd.Series(values, index=['st/dist','name','party']), ignore_index=True)
        
        df['dist'] = df['st/dist'].map(lambda x: int(x.split()[-1]) if x.split()[-1].isnumeric() else 98) #'at-large'
        df['st'] = df['st/dist'].map(abbreviate)
        df['party'] = df['party'].map(lambda x: x[0:3].upper())

    return df

def url_nm(name): return ''.join([i +'+' for i in name.split()]) #create FEC url name search string

def ids(x): # FEC search request to get candidate's ID
    result = BeautifulSoup(requests.get('https://www.fec.gov/search/?query=%s&type=candidates' %x).text,
                         'html.parser').find_all(class_='post__path t-small t-sans')
    return result[0].text.replace('<div>', '').split('/')[3] if result !=[] else 'no result'

def wiki_id(x): #iterate over df apply FEC search for each district's winner
    for i in range(len(x)):
        x.iat[i,5] = ids(url_nm(x.iat[i,3]))
    return x
    
    

In [6]:
 wk1 = wiki_id(wiki()) #create wiki data frame

In [7]:
 wk1.to_csv('wiki_fec_csv', index = None, header=True) #save to csv

# 3. Data Cleaning

In [10]:
pp = pd.read_csv('/.../Desktop/propub_csv_final') # Propublica data frame
wk = pd.read_csv('/.../Desktop/wiki_fec_csv')     # Wikipedia data frame

The first step is to go through the list of Propublica candidates and determine who won and who lost. The following for loop matches Propublica candidates to winning candidates from Wikipedia. For each Propublica candidate, look up their state and district in the Wikipedia table, a winner will match on the name or the FEC ID.

In [11]:
# iterate over every propublica candidate, did they win or lose?
for i in range(len(pp)):
    match = wk[(wk['st']==pp.iloc[i,0]) & (wk['dist']==pp.iloc[i,1])] #find candidate's state/district in the wiki df
    if match.empty: pp.iat[i, 7] = False #accounts for incorrect districts in Propublica data
    elif match['name'].values == 'vacant': pp.iat[i, 7] = False # there are currently 5 vacant seats
    else: 
        pp.iat[i, 7]= ((pp.iloc[i,2].split(',')[0] == match['name'].values[0].split(' ')[-1]) 
        or (pp.iloc[i,3] == match.id.values[0])) # match on name or FEC ID

Wikipedia says there are 435 members of congress. How many do we have?

In [12]:
print(len(pp[pp['win']==1.0]))

438


We have too many, something is off. Maybe there are duplicate winners? We can filter the data frame to show all winners, then group by state and distict, counting the number of winners. There should be one and only one for each district. We can filter again returning any districts with a winner count of 2.  

In [13]:
pp_dup = pp[pp['win']==1.0].groupby(['st', 'dist'], as_index = False).agg({'win':['count']})
pp_dup = pp_dup[pp_dup['win']['count']==2]

In [14]:
print(pp_dup)

     st dist   win
             count
45   CA   25     2
56   CA   36     2
62   CA   42     2
70   CA   50     2
130  IA    2     2
175  LA    5     2
191  MD    6     2
192  MD    7     2
266  NV    3     2
331  PA   12     2


There are duplicates. Let's print out the records.

In [15]:
for i in range(len(pp_dup)):
    print(pp[(pp['st']==ppd.iloc[i,0]) & (pp['dist']==ppd.iloc[i,1]) & (pp['win']==1.0)])

     st  dist             name         id party    total     pac  win
270  CA    25     garcia, mike  H0CA25162   REP   358113    8893  1.0
369  CA    25  garcia, michael  H0CA25105   REP  9348402  897272  1.0
     st  dist            name         id party    total     pac  win
173  CA    36      ruiz, raul  H0CA36177   REP    12039       0  1.0
504  CA    36  ruiz, raul dr.  H2CA36439   DEM  2428672  968325  1.0
     st  dist          name         id party    total     pac  win
180  CA    42  calvert, ken  H0CA42209   REP        0       0  1.0
505  CA    42  calvert, ken  H2CA37023   REP  1386747  611550  1.0
     st  dist           name         id party     total     pac  win
278  CA    50  issa, darrell  H0CA50178   REP  10196220  414060  1.0
300  CA    50  issa, darrell  H0CA48024   REP         0       0  1.0
      st  dist                            name         id party    total     pac  win
1254  IA     2  miller-meeks, mariannette jane  H0IA02180   REP   748422  110770  1.0
126

Examining the records...

Duplicates with $0 donations:

    2673  PA    12  keller, fred  H9PA12018
    1872  NV     3  lee, susie  H8NV03200 
    300  CA    50  issa, darrell  H0CA48024
    180  CA    42  calvert, ken  H0CA42209

Opposing candidate with nearly identical name:

    173  CA    36      ruiz, raul  H0CA36177
    
Julia Letlow won special election to replace deceased husband:

    1365  LA     5        letlow, julia  H2LA05126
    
No record found on FEC website for:

    1411  MD     7  mfume, kweisi  H0MD07114
    1406  MD     6  trone, david  H8MD06168
    1035  IA     2  miller-meeks, mariannette jane  H0IA02180
    289  CA    25     garcia, mike  H0CA25162


438 minus 10 duplicates = 428 members. Now we are short.


Is there a winner for every district listed on Wikipedia? For each state and district of the wikipedia data frame is there a candidate in the propublica data frame with the same state and district who won?

In [17]:
wk['match'] = wk.apply(lambda i: 
                         not (pp[(pp['st']==i.st) & (pp['dist']==i.dist) & (pp['win'] == 1.0)]).empty, axis= 1)

In [18]:
print(wk[wk['match']==False])

     st  dist       st/dist              name party         id  match
106  FL    20    Florida 20            vacant   VAC  no result  False
174  LA     2   Louisiana 2            vacant   VAC  no result  False
254  NM     1  New Mexico 1            vacant   VAC  no result  False
308  OH    11       Ohio 11            vacant   VAC  no result  False
366  TX     6       Texas 6            vacant   VAC  no result  False
384  TX    24      Texas 24    beth van duyne   REP  no result  False
394  TX    34      Texas 34  filemon vela jr.   DEM  no result  False


438 minus 10 duplicates + 5 vacancies + 2 missing = 435

The CSV file was edited by hand to correct for the 10 duplicate results and 2 missing candidates.

In [19]:
propub_winners_csv = pp.sort_values((['st', 'dist']), ascending=True)
propub_winners_csv.to_csv('propub_winners_csv', index = None, header=True) # save to csv for editing

In [20]:
win = pd.read_csv('/.../Desktop/propub_winners_csv_edit.csv') # open edited version

In [21]:
print(len(win[win['win']==1.0]))

430
